# Prorrata ERNC
Este programa tiene por objetivo realizar el re-calculo de curtailment para el sistema, en base a metodología propuesta por la NTCyO.

## LECTURA DE DATOS
Los datos deben ser extraidos desde el accdb en potencia neta (no por la potencia, sino por los marginales no truncados). La lista de datos que se deben extraer son:
1. Generación de cada central.
2. Perfil de generación de cada central.
3. Barra asociada a cada central.
4. Costos marginales para cada.
5. Curtailment por central (quizas por barra es suficiente).
6. Potencia máxima.
7. Generación disponible.
8. Estado de operación.

Para la lectura, a modo de determinar la mejor query al sistema, sin tener que lidiar con los problemas de MS Access, se cargan las tablas en DuckDB y se utiliza jupysql para probar SQL.

In [6]:
import polars as pl
import duckdb as duck

from pathlib import Path
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy import (
    engine,
    create_engine,
    inspect
)

path_prg = Path(r"../data/Model PRGdia_Full_Definitivo Solution.accdb").absolute()

if not path_prg.exists():
    raise ValueError(f"Path: {path_prg} does not exists.")

connection_string = (
    r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};"
    rf"DBQ={path_prg.as_posix()};"
    r"ExtendedAnsiSQL=1;"
)
connection_url = engine.URL.create(
    "access+pyodbc",
    query={"odbc_connect": connection_string}
)

try:
    prg_engine = create_engine(connection_url)
    tables = inspect(prg_engine).get_table_names()

    conn = duck.connect("PCP.duckdb")
    #conn.execute("CREATE SCHEMA IF NOT EXISTS bronze")

    for table in tables:
        print(f"trabajando en tabla: {table}...")
        df = pl.read_database(query=f"SELECT * FROM {table}", connection=prg_engine)
        conn.execute(f"CREATE OR REPLACE TABLE {table} AS SELECT * FROM df")

except SQLAlchemyError as e:
    print(f"Error: {e}")

finally:
    conn.close()
    prg_engine.dispose()

trabajando en tabla: t_attribute...
trabajando en tabla: t_attribute_data...
trabajando en tabla: t_band...
trabajando en tabla: t_category...
trabajando en tabla: t_class...
trabajando en tabla: t_class_group...
trabajando en tabla: t_collection...
trabajando en tabla: t_config...
trabajando en tabla: t_custom_column...
trabajando en tabla: t_data_0...
trabajando en tabla: t_data_1...
trabajando en tabla: t_data_2...
trabajando en tabla: t_data_3...
trabajando en tabla: t_data_4...
trabajando en tabla: t_data_6...
trabajando en tabla: t_data_7...
trabajando en tabla: t_data_current...
trabajando en tabla: t_key...
trabajando en tabla: t_key_index...
trabajando en tabla: t_membership...
trabajando en tabla: t_memo_object...
trabajando en tabla: t_model...
trabajando en tabla: t_object...
trabajando en tabla: t_object_meta...
trabajando en tabla: t_period_0...
trabajando en tabla: t_period_1...
trabajando en tabla: t_period_2...
trabajando en tabla: t_period_3...
trabajando en tabla: t_

## REVISIÓN DUCKDB
Con la data carga en la base de datos, empezamos a mirar como armar la mejor query

In [7]:
# Esto es para carga la extensión y leer la base de datos.
import duckdb

conn_pcp = duckdb.connect("pcp.duckdb")

# load de la extensión para sql
%load_ext sql
%sql conn_pcp --alias duck

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [ ]:
# Esto es para cerrar las conexiones, usarlo al terminar de revisar
%sql --close duck
conn.close()

In [27]:
%%sql
SELECT 
    t_child.name AS Central,
    t_parent.class_id AS Class_Id,
    t_category.name AS Category,
    t_property.name AS Property,
    t_period_0.datetime AS Fecha,
    t_data_0.key_id AS data_key,
    t_data_0.period_id AS data_period,
    t_data_0.value AS Valor,
FROM ((((((((t_membership
INNER JOIN t_collection ON t_membership.collection_id = t_collection.collection_id)
INNER JOIN t_object AS t_parent ON t_membership.parent_object_id = t_parent.object_id)
INNER JOIN t_object AS t_child ON t_membership.child_object_id = t_child.object_id)
INNER JOIN t_property ON t_collection.collection_id = t_property.collection_id)
INNER JOIN t_key ON t_membership.membership_id = t_key.membership_id AND t_property.property_id = t_key.property_id)
INNER JOIN t_data_0 ON t_key.key_id = t_data_0.key_id)
INNER JOIN t_phase_3 ON t_data_0.period_id = t_phase_3.period_id)
INNER JOIN t_period_0 ON t_phase_3.interval_id = t_period_0.interval_id)
INNER JOIN t_category ON t_child.category_id = t_category.category_id
WHERE t_collection.collection_id = 1 AND t_property.property_id IN (1, 28, 200, 219) AND t_category.category_id IN (95, 96, 99, 100)

Running query in 'duck'

Central,Class_Id,Category,Property,Fecha,data_key,data_period,Valor
EL_MAITEN_EO,1,Wind Farms,Max Capacity,2024-01-11 00:00:00,10428,1,9.0
EL_MAITEN_EO,1,Wind Farms,Max Capacity,2024-01-11 01:00:00,10428,2,9.0
EL_MAITEN_EO,1,Wind Farms,Max Capacity,2024-01-11 02:00:00,10428,3,9.0
EL_MAITEN_EO,1,Wind Farms,Max Capacity,2024-01-11 03:00:00,10428,4,9.0
EL_MAITEN_EO,1,Wind Farms,Max Capacity,2024-01-11 04:00:00,10428,5,9.0
EL_MAITEN_EO,1,Wind Farms,Max Capacity,2024-01-11 05:00:00,10428,6,9.0
EL_MAITEN_EO,1,Wind Farms,Max Capacity,2024-01-11 06:00:00,10428,7,9.0
EL_MAITEN_EO,1,Wind Farms,Max Capacity,2024-01-11 07:00:00,10428,8,9.0
EL_MAITEN_EO,1,Wind Farms,Max Capacity,2024-01-11 08:00:00,10428,9,9.0
EL_MAITEN_EO,1,Wind Farms,Max Capacity,2024-01-11 09:00:00,10428,10,9.0


In [76]:
%%sql
WITH node_obj AS (
    SELECT 
        t_object.object_id AS node_id,
        t_object.name AS node,
    FROM t_object
    INNER JOIN t_class ON t_object.class_id = t_class.class_id
    WHERE t_class.name = 'Node'
), gen_obj AS (
    SELECT 
        t_object.object_id AS gen_id,
        t_object.name AS generator,
    FROM t_object
    INNER JOIN t_class ON t_object.class_id = t_class.class_id
    WHERE t_class.name = 'Generator' AND t_object.category_id IN (95, 96, 99, 100)
)

SELECT
    node_obj.node,
    gen_obj.generator,
FROM t_membership
INNER JOIN node_obj ON t_membership.child_object_id = node_obj.node_id
INNER JOIN gen_obj ON t_membership.parent_object_id = gen_obj.gen_id
WHERE t_membership.collection_id = 12

Running query in 'duck'

node,generator
Andes220,SOL_DEL_NORTE_ANDES_FV
Angamos220,TALLADO_FV
Arica066,PAMPA_CAMARONES_FV
Cachiyuyal220,PAMPA_SOLAR_NORTE_FV
Capricornio110,CAPRICORNIO_FV
Cardones110,VALLE_SOLAR_OESTE_FV
Cardones220,VALLE_ESCONDIDO_FV
Condores220,WILLKA_FV
CPinto220,SAN_ANDRES_FV
Crucero220,LAS_SALINAS_FV


In [74]:
(
    result.PolarsDataFrame()
    .write_csv("node_gen_template2.csv")
)